In [1]:
#%pip install --upgrade torch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils_data
from torch.autograd import Variable

from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

import pandas as pd
import numpy as np

import pickle
import json
import os
import warnings
import datetime

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
Tesla V100-PCIE-16GB


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"{torch.cuda.device_count()} GPU(s) available and will be used.\nGPU name(s): {[torch.cuda.get_device_name(n) for n in range(torch.cuda.device_count())]}")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

1 GPU(s) available and will be used.
GPU name(s): ['Tesla V100-PCIE-16GB']


In [5]:
RANDOM_STATE = 42

In [6]:
# Instantiate a dataframe on first dataset to get columns
nfl_df = pd.read_csv('../../dataset_generators/datasets/0_train_normalized.csv')
nfl_df

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,...,fantasy_points_r_avg_5_lag_1,fantasy_points_r_avg_8_lag_1,opp_team_dst_fp_r_avg_1_lag_1,opp_team_dst_fp_r_avg_3_lag_1,opp_team_dst_fp_r_avg_5_lag_1,opp_team_dst_fp_r_avg_8_lag_1,opp_team_dst_fp_rank_r_avg_1_lag_1,opp_team_dst_fp_rank_r_avg_3_lag_1,opp_team_dst_fp_rank_r_avg_5_lag_1,opp_team_dst_fp_rank_r_avg_8_lag_1
0,0,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,4,REG,...,0.123524,0.123524,0.162162,0.315405,0.315405,0.315405,0.903226,0.467742,0.467742,0.467742
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,16,REG,...,0.114039,0.114039,0.405405,0.288378,0.302703,0.300541,0.306452,0.586129,0.525806,0.501935
2,2,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,17,REG,...,0.161678,0.161678,0.243243,0.270270,0.270270,0.260000,0.709677,0.634516,0.600000,0.586774
3,3,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,4,REG,...,0.197934,0.197934,0.054054,0.180270,0.180270,0.180270,0.983871,0.758065,0.758065,0.758065
4,4,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,11,REG,...,0.441821,0.441821,0.270270,0.270270,0.308108,0.337838,0.354839,0.457097,0.451613,0.445484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48770,48770,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,9,REG,...,0.155565,0.154089,0.189189,0.351351,0.329730,0.378378,0.741935,0.333226,0.367742,0.343226
48771,48771,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,12,REG,...,0.230185,0.196037,0.270270,0.207297,0.281081,0.300541,0.338710,0.645161,0.519355,0.480000
48772,48772,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,13,REG,...,0.216695,0.196037,0.324324,0.396486,0.351351,0.388649,0.354839,0.166774,0.309677,0.254194
48773,48773,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,14,REG,...,0.214165,0.196880,0.513514,0.459459,0.345946,0.364865,0.129032,0.177419,0.425806,0.377097


In [7]:
# Get normalized columns
y = ['fantasy_points']
X = [col for col in nfl_df.columns if col not in ('player_id'
                                                        , 'player_name'
                                                        , 'player_display_name'
                                                        , 'position','position_group'
                                                        , 'headshot_url'
                                                        , 'season'
                                                        , 'week'
                                                        , 'season_type'
                                                        , 'team'
                                                        , 'opponent_team'
                                                        , 'fantasy_points'
                                                        , 'Unnamed: 0')]
del nfl_df

In [8]:
nfl_df = pd.read_csv('../../dataset_generators/datasets/0_train_pca.csv')
nfl_df

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,...,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,fantasy_points
0,0,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,4,REG,...,-0.002146,0.005481,-0.052914,-0.047235,-0.032786,-0.021296,-0.043473,-0.430547,-0.039748,-0.20
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,16,REG,...,0.004413,0.031793,-0.008367,-0.022133,0.001059,-0.050775,-0.044723,0.517862,-0.022523,7.04
2,2,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,17,REG,...,0.237716,-0.098981,-0.007436,0.217254,0.649703,3.571516,-0.360357,-0.098601,0.170117,9.40
3,3,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,4,REG,...,-0.040106,-0.041452,-0.099212,-0.047098,-0.119749,-2.255649,0.232299,-0.543449,0.557499,15.38
4,5,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2015,11,REG,...,-0.025855,0.029912,-0.029561,0.055859,0.202644,0.288437,-0.131221,0.460260,0.242840,13.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48770,60963,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,9,REG,...,0.027369,0.024282,-0.016385,0.064106,-0.041373,0.023893,-0.004612,-0.229813,0.021283,11.50
48771,60965,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,12,REG,...,0.040545,0.025307,-0.039539,0.200897,-0.080051,-0.005432,-0.048712,1.058756,-0.056765,1.80
48772,60966,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,13,REG,...,0.012423,0.106695,0.056796,0.100761,-0.037281,-0.044826,-0.593010,-0.323484,0.035053,2.00
48773,60967,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2024,14,REG,...,0.024793,0.091568,0.048286,0.080314,-0.008332,-0.039124,-0.354176,-0.783789,0.043358,1.90


In [9]:
# Get pca columns
y_pca = ['fantasy_points']
X_pca = [col for col in nfl_df.columns if col not in ('player_id'
                                                        , 'player_name'
                                                        , 'player_display_name'
                                                        , 'position','position_group'
                                                        , 'headshot_url'
                                                        , 'season'
                                                        , 'week'
                                                        , 'season_type'
                                                        , 'team'
                                                        , 'opponent_team'
                                                        , 'fantasy_points'
                                                        , 'Unnamed: 0')]
del nfl_df

In [10]:
# Function to get the 3 error metrics we decided to use
def get_errors(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, rmse, r2

In [11]:
def get_checkpoint(path):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
        return data['dataset'], data['hls'], data['act'], data['mi'], data['lri']
    except:
        return None, None, None, None, None

In [12]:
# MLP Regressor but using PyTorch in order to use GPUs
# Code taken and tweaked from a question on pytorch forums: https://discuss.pytorch.org/t/sklearn-model-to-pytorch-model/33532
class relu_MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(relu_MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out

In [13]:
class tanh_MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(tanh_MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        out = self.fc3(x)
        return out

In [14]:
def fit_model(hls,mi,act,lri,x,y):
    
    start_time = datetime.datetime.now()

    input_size = len(x.columns)
    hidden_size1 = hls[0]
    hidden_size2 = hls[1]
    output_size = 1
    
    x_tensor = torch.tensor(x.values, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y.values.reshape(-1, 1), dtype=torch.float32).to(device)
    
    training_samples = utils_data.TensorDataset(x_tensor, y_tensor)
    data_loader_trn = utils_data.DataLoader(training_samples, batch_size=200, shuffle=True, drop_last=False)
    
    if act == 'relu':
        model = relu_MLP(input_size = input_size, hidden_size1 = hidden_size1,
            hidden_size2 = hidden_size2, output_size = output_size).to(device)
    elif act == 'tanh':
        model = tanh_MLP(input_size = input_size, hidden_size1 = hidden_size1,
            hidden_size2 = hidden_size2, output_size = output_size).to(device)
    else:
        raise Exception('Unsupported activation function!')
    
    model = nn.DataParallel(model)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = lri, weight_decay=0.0001)
    loss_fct = nn.MSELoss()
    
    for epoch in range(mi):
            cum_loss = 0
            for batch_idx, (x, y) in enumerate(data_loader_trn):

                tr_x, tr_y = x.float().to(device), y.float().to(device)

                pred = model(tr_x)
                loss = loss_fct(pred, tr_y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                cum_loss += loss.item()

#             if epoch % (mi/10) == 0:
#                  print ('Epoch [%d/%d], Loss: %.4f' 
#                            %(epoch+1, mi, cum_loss))
#     print(f"Model trained in {datetime.datetime.now()-start_time}")
    return model

In [15]:
# hyper parameters to iterate over
hidden_layer_sizes = [(100,100),(500,500),(1000,1000)]
max_iters = [100,500,1000]
activations = ['relu','tanh']
learning_rate_inits = [0.0001,0.001,0.01]

In [18]:
# Normalized feature set model generation

# Get current time to track model time
current_time = datetime.datetime.now()

# Number of models tracking
num = 1
total_models = len(range(5)) * len(hidden_layer_sizes) * len(activations) * len(max_iters) * len(learning_rate_inits)

# Checkpoint file processing
checkpoint_file_path = './models/checkpoint.json'
checkpoint_dataset, checkpoint_hls, checkpoint_act, checkpoint_mi, checkpoint_lri = get_checkpoint(checkpoint_file_path)
try:
    checkpoint_hls = tuple(checkpoint_hls)
    print('Checkpoint file found')
    num += 1
except:
    print('No checkpoint')
if checkpoint_dataset == None and checkpoint_hls == None and checkpoint_act == None and checkpoint_mi == None and checkpoint_lri == None:
    dataset_flag, hls_flag, act_flag, mi_flag, lri_flag = 1,1,1,1,1
else:
    dataset_flag, hls_flag, act_flag, mi_flag, lri_flag = 0,0,0,0,0

# Iterate over everything, fit the model, and export to pickle
for n in range(5):
    if n == checkpoint_dataset and dataset_flag == 0:
        dataset_flag = 1
    elif dataset_flag != 1:
        num += 54
        continue
    dataset_path = f'../../dataset_generators/datasets/{n}_train_normalized.csv'
    data = pd.read_csv(dataset_path)
    X_train = data[X]
    y_train = data[y]
    for hls in hidden_layer_sizes:
        if hls == checkpoint_hls and hls_flag == 0:
            hls_flag = 1
        elif hls_flag != 1:
            num += 18
            continue
        for act in activations:
            if act == checkpoint_act and act_flag == 0:
                act_flag = 1
            elif act_flag != 1:
                num += 9
                continue
            for mi in max_iters:
                if mi == checkpoint_mi and mi_flag == 0:
                    mi_flag = 1
                elif mi_flag != 1:
                    num += 3
                    continue
                for lri in learning_rate_inits:
                    if lri == checkpoint_lri and lri_flag == 0:
                        lri_flag = 1
                    elif lri_flag == 1:
                        mlp = fit_model(hls,mi,act,lri,X_train,y_train)
                        with open(f'./models/{n}_{hls[0]}-{hls[1]}_{act}_{mi}_{lri}_norm.pickle', 'wb') as f:
                            pickle.dump(mlp, f)
                            print(f'Model {num}/{total_models} created "./models/{n}_{hls[0]}-{hls[1]}_{act}_{mi}_{lri}_norm.pickle" - elapsed time: {datetime.datetime.now()-current_time}')
                            num += 1
                        checkpoint = {'dataset':n,'hls':hls,'act':act,'mi':mi,'lri':lri}
                        with open(checkpoint_file_path, 'w') as f:
                            json.dump(checkpoint, f, indent=4)
                    elif lri_flag != 1:
                        num += 1

Checkpoint file found


In [19]:
# PCA feature set model generation

# Get current time to track model time
current_time = datetime.datetime.now()

# Number of models tracking
num = 1
total_models = len(range(5)) * len(hidden_layer_sizes) * len(activations) * len(max_iters) * len(learning_rate_inits)

# Checkpoint file processing
checkpoint_file_path = './models/pca_checkpoint.json'
checkpoint_dataset, checkpoint_hls, checkpoint_act, checkpoint_mi, checkpoint_lri = get_checkpoint(checkpoint_file_path)
try:
    checkpoint_hls = tuple(checkpoint_hls)
    print('Checkpoint file found')
    num += 1
except:
    print('No checkpoint')
if checkpoint_dataset == None and checkpoint_hls == None and checkpoint_act == None and checkpoint_mi == None and checkpoint_lri == None:
    dataset_flag, hls_flag, act_flag, mi_flag, lri_flag = 1,1,1,1,1
else:
    dataset_flag, hls_flag, act_flag, mi_flag, lri_flag = 0,0,0,0,0

# Iterate over everything, fit the model, and export to pickle
for n in range(5):
    if n == 1: # Somehow I did not create the 0_ pca models correctly... but the rest seem to have been created correctly
        break
    if n == checkpoint_dataset and dataset_flag == 0:
        dataset_flag = 1
    elif dataset_flag != 1:
        num += 54
        continue
    dataset_path = f'../../dataset_generators/datasets/{n}_train_pca.csv'
    data = pd.read_csv(dataset_path)
    X_train = data[X_pca]
    y_train = data[y_pca]
    for hls in hidden_layer_sizes:
        if hls == checkpoint_hls and hls_flag == 0:
            hls_flag = 1
        elif hls_flag != 1:
            num += 18
            continue
        for act in activations:
            if act == checkpoint_act and act_flag == 0:
                act_flag = 1
            elif act_flag != 1:
                num += 9
                continue
            for mi in max_iters:
                if mi == checkpoint_mi and mi_flag == 0:
                    mi_flag = 1
                elif mi_flag != 1:
                    num += 3
                    continue
                for lri in learning_rate_inits:
                    if lri == checkpoint_lri and lri_flag == 0:
                        lri_flag = 1
                    elif lri_flag == 1:
                        model = fit_model(hls,mi,act,lri,X_train,y_train)
                        with open(f'./models/{n}_{hls[0]}-{hls[1]}_{act}_{mi}_{lri}_pca.pickle', 'wb') as f:
                            pickle.dump(model, f)
                            print(f'Model {num}/{total_models} created "./models/{n}_{hls[0]}-{hls[1]}_{act}_{mi}_{lri}_pca.pickle" - elapsed time: {datetime.datetime.now()-current_time}')
                            num += 1
                        checkpoint = {'dataset':n,'hls':hls,'act':act,'mi':mi,'lri':lri}
                        with open(checkpoint_file_path, 'w') as f:
                            json.dump(checkpoint, f, indent=4)

Checkpoint file found


In [64]:
# Sort the models by their characteristics so we can get the mean of their kfolds performance
models = []
for model in os.listdir('./models'):
    if 'checkpoint' not in model and 'model_metrics' not in model:
        models.append(model)
models.sort(key=lambda k: (k.split('_')[-1]
                                 ,k.split('_')[-5].split('-')[0]
                                 ,k.split('_')[-5].split('-')[1]
                                 ,k.split('_')[-4]
                                 ,k.split('_')[-3]
                                 ,k.split('_')[-2]
                                 ,k.split('_')[-6]))

if len(models) != 540:
    raise Exception('Not all models created!')
else:
    print(f'All {len(models)} created')

# Instantiate the dictionary that will be converted into a dataframe
model_metric_dict = {}

#print(len([model for model in models if 'pca.pickle' in model]))  #270 good
#print(len([model for model in models if 'norm.pickle' in model])) #270 good

# Loop through models
for idx, model in enumerate(models):
    # Get arbitrary key that will be used as index in dataframe if all 4 models have been predicted or if we are in iteration 0
    if idx == 0 or (idx+1)%5 == 0:
        
        
        # If it isn't first iteration, then mean the list
        if idx != 0:
            model_metric_dict[dict_key]['mean_absolute_error_avg'] = np.mean(np.array(model_metric_dict[dict_key]['mean_absolute_error']))
            model_metric_dict[dict_key]['root_mean_squared_error_avg'] = np.mean(np.array(model_metric_dict[dict_key]['root_mean_squared_error']))
            model_metric_dict[dict_key]['r_squared_avg'] = np.mean(np.array(model_metric_dict[dict_key]['r_squared']))

            model_metric_dict[dict_key]['mean_absolute_error_std'] = np.std(np.array(model_metric_dict[dict_key]['mean_absolute_error']))
            model_metric_dict[dict_key]['root_mean_squared_error_std'] = np.std(np.array(model_metric_dict[dict_key]['root_mean_squared_error']))
            model_metric_dict[dict_key]['r_squared_std'] = np.std(np.array(model_metric_dict[dict_key]['r_squared']))
        
        
        dict_key = (idx+1)/5
        
        # Get the parameters of the model
        hls = model.split('_')[-5]
        act = model.split('_')[-4]
        mi = model.split('_')[-3]
        lri = model.split('_')[-2]
        mae = []
        rmse = []
        r2 = []

        # Add them to the dictionary
        model_metric_dict[dict_key] = {'hidden_layer_size':hls
                   ,'activation_function':act
                   ,'max_iters':mi
                   ,'learning_rate_init':lri
                   ,'mean_absolute_error':[]
                   ,'root_mean_squared_error':[]
                   ,'r_squared':[]}

        # Add the feature type attribute to the record
        if 'norm.pickle' in model: 
            model_metric_dict[dict_key]['feature_type'] = 'normalized'
        elif 'pca.pickle' in model:
            model_metric_dict[dict_key]['feature_type'] = 'pca'        
        else:
            raise Exception(f'Model name {model} is incorrectly named!')
            
    # Load the model    
    with open(f'./models/{model}', 'rb') as f:
        #print(model)
        mlp = pickle.load(f)
        mlp.eval()
    
    # Get the dataset number to pull the test version
    dataset_num = model.split('_')[-6]
    if 'norm.pickle' in model:
        data_df = pd.read_csv(f'../../dataset_generators/datasets/{dataset_num}_test_normalized.csv')
        # Get X and y
        X_test = data_df[X]
        y_test = data_df[y]
    elif 'pca.pickle' in model:
        data_df = pd.read_csv(f'../../dataset_generators/datasets/{dataset_num}_test_pca.csv')
        X_test = data_df[X_pca]
        y_test = data_df[y_pca]
    else:
        raise Exception(f'Model name {model} is incorrectly named!')
    
    # Get the predictions for y
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    with torch.no_grad():
        try:
            y_pred = mlp(X_test_tensor)
            y_pred = y_pred.cpu().numpy()
        except Exception as e:
            print(f"Model name: {model}\nError: {str(e)}")
            #raise Exception("Model prediction failed!")
    
    # Get errors and append them to their designated lists
    mae, rmse, r2 = get_errors(y_test, y_pred)
    model_metric_dict[dict_key]['mean_absolute_error'].append(mae)
    model_metric_dict[dict_key]['root_mean_squared_error'].append(rmse)
    model_metric_dict[dict_key]['r_squared'].append(r2)
    
    
# The for loop will end up skipping the last dict_key's means
# So we will do it here
model_metric_dict[dict_key]['mean_absolute_error_avg'] = np.mean(np.array(model_metric_dict[dict_key]['mean_absolute_error']))
model_metric_dict[dict_key]['root_mean_squared_error_avg'] = np.mean(np.array(model_metric_dict[dict_key]['root_mean_squared_error']))
model_metric_dict[dict_key]['r_squared_avg'] = np.mean(np.array(model_metric_dict[dict_key]['r_squared']))

model_metric_dict[dict_key]['mean_absolute_error_std'] = np.std(np.array(model_metric_dict[dict_key]['mean_absolute_error']))
model_metric_dict[dict_key]['root_mean_squared_error_std'] = np.std(np.array(model_metric_dict[dict_key]['root_mean_squared_error']))
model_metric_dict[dict_key]['r_squared_std'] = np.std(np.array(model_metric_dict[dict_key]['r_squared']))

# Backup the dictionary
with open('./models/model_metrics.json', 'w') as f:
    json.dump(model_metric_dict, f, indent=4)

All 540 created


In [65]:
# Create pandas dataframe out of model scores
try:
    model_df = pd.DataFrame.from_dict(model_metric_dict, orient='index')
except:
    try:
        with open('./models/model_metrics.json', 'r') as f:
            model_metric_dict = json.load(f)
            model_df = pd.DataFrame.from_dict(model_metric_dict, orient='index')
    except:
        raise Exception('No dictionary or file!')

In [66]:
# Models sorted by mean absolute error
model_df.sort_values(by=['mean_absolute_error_avg'], ascending=True).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,tanh,100,0.001,"[3.750624656677246, 3.7519431114196777, 3.7525...","[5.2765421867370605, 5.40845251083374, 5.45955...","[0.410799503326416, 0.400351881980896, 0.40192...",normalized,3.791980,5.389775,0.403647,0.073207,0.072339,0.003668
1,100-100,tanh,100,0.0001,"[3.735905647277832, 3.9204981327056885, 3.7857...","[5.274491310119629, 5.377488613128662, 5.42190...","[0.41125738620758057, 0.40719830989837646, 0.4...",normalized,3.826245,5.370486,0.407880,0.078870,0.059915,0.002819
2,100-100,relu,500,0.01,"[3.8246326446533203, 3.861290216445923, 3.8463...","[5.491226673126221, 5.3739423751831055, 5.4163...","[0.36187905073165894, 0.4079798460006714, 0.41...",normalized,3.829851,5.412269,0.398365,0.021622,0.061261,0.018419
3,100-100,relu,100,0.0001,"[3.8208563327789307, 3.8558013439178467, 3.890...","[5.367282867431641, 5.416212558746338, 5.43947...","[0.40944623947143555, 0.41137897968292236, 0.4...",normalized,3.843696,5.385337,0.409060,0.032374,0.046622,0.001530
4,100-100,tanh,100,0.01,"[3.8080341815948486, 3.810051918029785, 3.8659...","[5.2921905517578125, 5.3774847984313965, 5.412...","[0.40729963779449463, 0.4071991443634033, 0.41...",normalized,3.845450,5.367951,0.408423,0.031556,0.052212,0.002276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1000-1000,relu,1000,0.0001,"[4.674513816833496, 5.069432258605957, 5.15864...","[6.735681056976318, 7.240201950073242, 7.38092...","[0.03987610340118408, -0.07461225986480713, -0...",normalized,5.013265,7.164054,-0.054145,0.173307,0.224019,0.054015
105,500-500,relu,1000,0.0001,"[4.580015659332275, 5.135223388671875, 5.13435...","[6.681125164031982, 7.3593220710754395, 7.4060...","[0.0553661584854126, -0.11026358604431152, -0....",pca,5.029695,7.244305,-0.078214,0.225367,0.282222,0.069938
106,500-500,tanh,500,0.0001,"[4.277072906494141, 5.295821189880371, 5.23113...","[6.112253189086914, 7.157207012176514, 7.13159...","[0.2093815803527832, -0.050116777420043945, -0...",pca,5.045109,6.937125,0.009878,0.385399,0.412518,0.101369
107,1000-1000,tanh,100,0.01,"[4.432693958282471, 5.395664691925049, 5.33677...","[5.790872573852539, 7.427653789520264, 7.33027...","[0.2903367877006531, -0.13097703456878662, -0....",pca,5.180923,7.069394,-0.032116,0.374695,0.640758,0.162417


In [67]:
# Models sorted by root mean squared error
model_df.sort_values(by=['root_mean_squared_error_avg'], ascending=True).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,tanh,100,0.01,"[3.8080341815948486, 3.810051918029785, 3.8659...","[5.2921905517578125, 5.3774847984313965, 5.412...","[0.40729963779449463, 0.4071991443634033, 0.41...",normalized,3.845450,5.367951,0.408423,0.031556,0.052212,0.002276
1,100-100,tanh,100,0.0001,"[3.735905647277832, 3.9204981327056885, 3.7857...","[5.274491310119629, 5.377488613128662, 5.42190...","[0.41125738620758057, 0.40719830989837646, 0.4...",normalized,3.826245,5.370486,0.407880,0.078870,0.059915,0.002819
2,100-100,relu,100,0.0001,"[3.8208563327789307, 3.8558013439178467, 3.890...","[5.367282867431641, 5.416212558746338, 5.43947...","[0.40944623947143555, 0.41137897968292236, 0.4...",normalized,3.843696,5.385337,0.409060,0.032374,0.046622,0.001530
3,500-500,tanh,1000,0.01,"[4.030289649963379, 3.878767967224121, 3.90477...","[5.342724800109863, 5.384793758392334, 5.43087...","[0.395926296710968, 0.4055866003036499, 0.4081...",normalized,3.896201,5.388226,0.403923,0.072769,0.046193,0.004209
4,500-500,tanh,100,0.0001,"[3.746528387069702, 3.9126462936401367, 3.8761...","[5.272403240203857, 5.401885986328125, 5.43469...","[0.4117234945297241, 0.40180712938308716, 0.40...",normalized,3.873061,5.388941,0.403805,0.068070,0.066967,0.005297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,500-500,relu,1000,0.0001,"[4.075437545776367, 5.07201623916626, 5.076261...","[5.921423435211182, 7.375552177429199, 7.26933...","[0.25797849893569946, -0.1151660680770874, -0....",normalized,4.845773,7.043102,-0.022764,0.390161,0.567833,0.141782
105,1000-1000,tanh,100,0.01,"[4.432693958282471, 5.395664691925049, 5.33677...","[5.790872573852539, 7.427653789520264, 7.33027...","[0.2903367877006531, -0.13097703456878662, -0....",pca,5.180923,7.069394,-0.032116,0.374695,0.640758,0.162417
106,1000-1000,relu,1000,0.0001,"[4.674513816833496, 5.069432258605957, 5.15864...","[6.735681056976318, 7.240201950073242, 7.38092...","[0.03987610340118408, -0.07461225986480713, -0...",normalized,5.013265,7.164054,-0.054145,0.173307,0.224019,0.054015
107,500-500,tanh,1000,0.0001,"[4.888935565948486, 5.3715596199035645, 5.3185...","[6.939290523529053, 7.30993127822876, 7.205881...","[-0.01904749870300293, -0.09541082382202148, -...",pca,5.232223,7.171531,-0.056030,0.174776,0.123885,0.027944


In [68]:
# Models sorted by r-squared
model_df.sort_values(by=['r_squared_avg'], ascending=False).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,relu,100,0.0001,"[3.8208563327789307, 3.8558013439178467, 3.890...","[5.367282867431641, 5.416212558746338, 5.43947...","[0.40944623947143555, 0.41137897968292236, 0.4...",normalized,3.843696,5.385337,0.409060,0.032374,0.046622,0.001530
1,100-100,tanh,100,0.01,"[3.8080341815948486, 3.810051918029785, 3.8659...","[5.2921905517578125, 5.3774847984313965, 5.412...","[0.40729963779449463, 0.4071991443634033, 0.41...",normalized,3.845450,5.367951,0.408423,0.031556,0.052212,0.002276
2,100-100,tanh,100,0.0001,"[3.735905647277832, 3.9204981327056885, 3.7857...","[5.274491310119629, 5.377488613128662, 5.42190...","[0.41125738620758057, 0.40719830989837646, 0.4...",normalized,3.826245,5.370486,0.407880,0.078870,0.059915,0.002819
3,500-500,tanh,1000,0.01,"[4.030289649963379, 3.878767967224121, 3.90477...","[5.342724800109863, 5.384793758392334, 5.43087...","[0.395926296710968, 0.4055866003036499, 0.4081...",normalized,3.896201,5.388226,0.403923,0.072769,0.046193,0.004209
4,500-500,tanh,100,0.0001,"[3.746528387069702, 3.9126462936401367, 3.8761...","[5.272403240203857, 5.401885986328125, 5.43469...","[0.4117234945297241, 0.40180712938308716, 0.40...",normalized,3.873061,5.388941,0.403805,0.068070,0.066967,0.005297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,500-500,relu,1000,0.0001,"[4.075437545776367, 5.07201623916626, 5.076261...","[5.921423435211182, 7.375552177429199, 7.26933...","[0.25797849893569946, -0.1151660680770874, -0....",normalized,4.845773,7.043102,-0.022764,0.390161,0.567833,0.141782
105,1000-1000,tanh,100,0.01,"[4.432693958282471, 5.395664691925049, 5.33677...","[5.790872573852539, 7.427653789520264, 7.33027...","[0.2903367877006531, -0.13097703456878662, -0....",pca,5.180923,7.069394,-0.032116,0.374695,0.640758,0.162417
106,1000-1000,relu,1000,0.0001,"[4.674513816833496, 5.069432258605957, 5.15864...","[6.735681056976318, 7.240201950073242, 7.38092...","[0.03987610340118408, -0.07461225986480713, -0...",normalized,5.013265,7.164054,-0.054145,0.173307,0.224019,0.054015
107,500-500,tanh,1000,0.0001,"[4.888935565948486, 5.3715596199035645, 5.3185...","[6.939290523529053, 7.30993127822876, 7.205881...","[-0.01904749870300293, -0.09541082382202148, -...",pca,5.232223,7.171531,-0.056030,0.174776,0.123885,0.027944


In [74]:
# It appears that the top models 5 are the ones that are normalized with a hidden layer of only 100-100 and 500-500
# The only model that is in the top 3 of all metrics is 
# 100-100	tanh	100	0.0001
model_df[(model_df['hidden_layer_size']=='100-100') 
         & (model_df['activation_function']=='tanh') 
         & (model_df['max_iters']=='100') 
         & (model_df['learning_rate_init']=='0.0001')
         & (model_df['feature_type']=='normalized')]

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
10.0,100-100,tanh,100,0.0001,"[3.735905647277832, 3.9204981327056885, 3.7857...","[5.274491310119629, 5.377488613128662, 5.42190...","[0.41125738620758057, 0.40719830989837646, 0.4...",normalized,3.826245,5.370486,0.40788,0.07887,0.059915,0.002819


In [ ]:
# What are the best pca models?

In [70]:
# PCA models sorted by mean absolute error
model_df[model_df['feature_type']=='pca'].sort_values(by=['mean_absolute_error_avg'], ascending=True).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,relu,500,0.01,"[4.159827709197998, 3.887394905090332, 3.94160...","[6.1093621253967285, 5.403135776519775, 5.4476...","[0.21012938022613525, 0.4015302062034607, 0.40...",pca,3.954768,5.558533,0.363011,0.108728,0.277856,0.076474
1,1000-1000,relu,500,0.01,"[3.9852826595306396, 3.9422714710235596, 4.030...","[5.817359924316406, 5.535642147064209, 5.62416...","[0.2838299870491028, 0.37181657552719116, 0.36...",pca,3.971972,5.625897,0.349567,0.048625,0.104704,0.033073
2,500-500,relu,500,0.01,"[4.185136318206787, 3.9506547451019287, 3.9811...","[6.165037631988525, 5.497476577758789, 5.55739...","[0.19566726684570312, 0.38044869899749756, 0.3...",pca,3.978086,5.647463,0.342789,0.109977,0.262095,0.073570
3,100-100,relu,100,0.0001,"[3.7945902347564697, 3.994060516357422, 4.0223...","[5.303995609283447, 5.5815911293029785, 5.7186...","[0.40465247631073, 0.36134469509124756, 0.3437...",pca,3.986046,5.584920,0.359594,0.099890,0.149583,0.023730
4,100-100,relu,100,0.01,"[3.850886583328247, 4.01019287109375, 4.033511...","[5.449075222015381, 5.637721538543701, 5.69956...","[0.37163811922073364, 0.34843504428863525, 0.3...",pca,3.994413,5.643157,0.346139,0.077863,0.101193,0.017270
5,100-100,tanh,100,0.0001,"[3.761805295944214, 4.110535144805908, 4.03998...","[5.310402870178223, 5.73422384262085, 5.743077...","[0.4032132625579834, 0.32593822479248047, 0.33...",pca,3.995280,5.645385,0.345573,0.125005,0.170768,0.029177
6,100-100,relu,100,0.001,"[4.05238151550293, 3.98183536529541, 3.9688556...","[5.5278825759887695, 5.584373474121094, 5.6686...","[0.3533312678337097, 0.36070793867111206, 0.35...",pca,4.005038,5.609391,0.354006,0.028853,0.057388,0.004561
7,100-100,relu,1000,0.01,"[4.234184741973877, 3.9727208614349365, 3.9516...","[6.22808313369751, 5.5338873863220215, 5.56418...","[0.17913252115249634, 0.37221473455429077, 0.3...",pca,4.006659,5.683085,0.334283,0.116875,0.275364,0.077635
8,1000-1000,relu,100,0.001,"[4.133289813995361, 4.016430854797363, 4.03817...","[6.03608512878418, 5.549175262451172, 5.602859...","[0.2289634346961975, 0.36874133348464966, 0.37...",pca,4.023656,5.690745,0.333790,0.070433,0.192106,0.053788
9,500-500,relu,100,0.001,"[4.143589496612549, 4.045867919921875, 3.94541...","[6.003452301025391, 5.651057243347168, 5.68735...","[0.23727774620056152, 0.34534889459609985, 0.3...",pca,4.041445,5.751719,0.319901,0.080664,0.135536,0.041916


In [71]:
# PCA models sorted by root mean squared error
model_df[model_df['feature_type']=='pca'].sort_values(by=['root_mean_squared_error_avg'], ascending=True).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,relu,500,0.01,"[4.159827709197998, 3.887394905090332, 3.94160...","[6.1093621253967285, 5.403135776519775, 5.4476...","[0.21012938022613525, 0.4015302062034607, 0.40...",pca,3.954768,5.558533,0.363011,0.108728,0.277856,0.076474
1,100-100,relu,100,0.0001,"[3.7945902347564697, 3.994060516357422, 4.0223...","[5.303995609283447, 5.5815911293029785, 5.7186...","[0.40465247631073, 0.36134469509124756, 0.3437...",pca,3.986046,5.584920,0.359594,0.099890,0.149583,0.023730
2,100-100,relu,100,0.001,"[4.05238151550293, 3.98183536529541, 3.9688556...","[5.5278825759887695, 5.584373474121094, 5.6686...","[0.3533312678337097, 0.36070793867111206, 0.35...",pca,4.005038,5.609391,0.354006,0.028853,0.057388,0.004561
3,1000-1000,relu,500,0.01,"[3.9852826595306396, 3.9422714710235596, 4.030...","[5.817359924316406, 5.535642147064209, 5.62416...","[0.2838299870491028, 0.37181657552719116, 0.36...",pca,3.971972,5.625897,0.349567,0.048625,0.104704,0.033073
4,100-100,relu,100,0.01,"[3.850886583328247, 4.01019287109375, 4.033511...","[5.449075222015381, 5.637721538543701, 5.69956...","[0.37163811922073364, 0.34843504428863525, 0.3...",pca,3.994413,5.643157,0.346139,0.077863,0.101193,0.017270
5,1000-1000,tanh,500,0.01,"[4.49981689453125, 3.860339879989624, 3.913384...","[6.113142013549805, 5.51239538192749, 5.551201...","[0.20915162563323975, 0.37708163261413574, 0.3...",pca,4.043021,5.643692,0.343960,0.232087,0.236090,0.067567
6,100-100,tanh,100,0.0001,"[3.761805295944214, 4.110535144805908, 4.03998...","[5.310402870178223, 5.73422384262085, 5.743077...","[0.4032132625579834, 0.32593822479248047, 0.33...",pca,3.995280,5.645385,0.345573,0.125005,0.170768,0.029177
7,500-500,relu,500,0.01,"[4.185136318206787, 3.9506547451019287, 3.9811...","[6.165037631988525, 5.497476577758789, 5.55739...","[0.19566726684570312, 0.38044869899749756, 0.3...",pca,3.978086,5.647463,0.342789,0.109977,0.262095,0.073570
8,100-100,relu,1000,0.01,"[4.234184741973877, 3.9727208614349365, 3.9516...","[6.22808313369751, 5.5338873863220215, 5.56418...","[0.17913252115249634, 0.37221473455429077, 0.3...",pca,4.006659,5.683085,0.334283,0.116875,0.275364,0.077635
9,1000-1000,relu,100,0.001,"[4.133289813995361, 4.016430854797363, 4.03817...","[6.03608512878418, 5.549175262451172, 5.602859...","[0.2289634346961975, 0.36874133348464966, 0.37...",pca,4.023656,5.690745,0.333790,0.070433,0.192106,0.053788


In [72]:
# PCA models sorted by r-squared
model_df[model_df['feature_type']=='pca'].sort_values(by=['r_squared_avg'], ascending=False).reset_index(drop=True)

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
0,100-100,relu,500,0.01,"[4.159827709197998, 3.887394905090332, 3.94160...","[6.1093621253967285, 5.403135776519775, 5.4476...","[0.21012938022613525, 0.4015302062034607, 0.40...",pca,3.954768,5.558533,0.363011,0.108728,0.277856,0.076474
1,100-100,relu,100,0.0001,"[3.7945902347564697, 3.994060516357422, 4.0223...","[5.303995609283447, 5.5815911293029785, 5.7186...","[0.40465247631073, 0.36134469509124756, 0.3437...",pca,3.986046,5.584920,0.359594,0.099890,0.149583,0.023730
2,100-100,relu,100,0.001,"[4.05238151550293, 3.98183536529541, 3.9688556...","[5.5278825759887695, 5.584373474121094, 5.6686...","[0.3533312678337097, 0.36070793867111206, 0.35...",pca,4.005038,5.609391,0.354006,0.028853,0.057388,0.004561
3,1000-1000,relu,500,0.01,"[3.9852826595306396, 3.9422714710235596, 4.030...","[5.817359924316406, 5.535642147064209, 5.62416...","[0.2838299870491028, 0.37181657552719116, 0.36...",pca,3.971972,5.625897,0.349567,0.048625,0.104704,0.033073
4,100-100,relu,100,0.01,"[3.850886583328247, 4.01019287109375, 4.033511...","[5.449075222015381, 5.637721538543701, 5.69956...","[0.37163811922073364, 0.34843504428863525, 0.3...",pca,3.994413,5.643157,0.346139,0.077863,0.101193,0.017270
5,100-100,tanh,100,0.0001,"[3.761805295944214, 4.110535144805908, 4.03998...","[5.310402870178223, 5.73422384262085, 5.743077...","[0.4032132625579834, 0.32593822479248047, 0.33...",pca,3.995280,5.645385,0.345573,0.125005,0.170768,0.029177
6,1000-1000,tanh,500,0.01,"[4.49981689453125, 3.860339879989624, 3.913384...","[6.113142013549805, 5.51239538192749, 5.551201...","[0.20915162563323975, 0.37708163261413574, 0.3...",pca,4.043021,5.643692,0.343960,0.232087,0.236090,0.067567
7,500-500,relu,500,0.01,"[4.185136318206787, 3.9506547451019287, 3.9811...","[6.165037631988525, 5.497476577758789, 5.55739...","[0.19566726684570312, 0.38044869899749756, 0.3...",pca,3.978086,5.647463,0.342789,0.109977,0.262095,0.073570
8,100-100,relu,1000,0.01,"[4.234184741973877, 3.9727208614349365, 3.9516...","[6.22808313369751, 5.5338873863220215, 5.56418...","[0.17913252115249634, 0.37221473455429077, 0.3...",pca,4.006659,5.683085,0.334283,0.116875,0.275364,0.077635
9,1000-1000,relu,100,0.001,"[4.133289813995361, 4.016430854797363, 4.03817...","[6.03608512878418, 5.549175262451172, 5.602859...","[0.2289634346961975, 0.36874133348464966, 0.37...",pca,4.023656,5.690745,0.333790,0.070433,0.192106,0.053788


In [73]:
# The best PCA model
model_df[(model_df['hidden_layer_size']=='100-100') 
         & (model_df['activation_function']=='relu') 
         & (model_df['max_iters']=='500') 
         & (model_df['learning_rate_init']=='0.01')
         & (model_df['feature_type']=='pca')]

,hidden_layer_size,activation_function,max_iters,learning_rate_init,mean_absolute_error,root_mean_squared_error,r_squared,feature_type,mean_absolute_error_avg,root_mean_squared_error_avg,r_squared_avg,mean_absolute_error_std,root_mean_squared_error_std,r_squared_std
63.0,100-100,relu,500,0.01,"[4.159827709197998, 3.887394905090332, 3.94160...","[6.1093621253967285, 5.403135776519775, 5.4476...","[0.21012938022613525, 0.4015302062034607, 0.40...",pca,3.954768,5.558533,0.363011,0.108728,0.277856,0.076474
